<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [31]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

Reading in and Exploring the dataset

In [32]:
# Importing the dataset
car_insurance = pd.read_csv('car_insurance.csv')

# Display basic info about the dataset
print(car_insurance.info())

# Check for missing values
print(car_insurance.isnull().sum())

# Display summary statistics
print(car_insurance.describe())

# Display the first few rows to get an overview
print(car_insurance.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   age                  10000 non-null  int64  
 2   gender               10000 non-null  int64  
 3   driving_experience   10000 non-null  object 
 4   education            10000 non-null  object 
 5   income               10000 non-null  object 
 6   credit_score         9018 non-null   float64
 7   vehicle_ownership    10000 non-null  float64
 8   vehicle_year         10000 non-null  object 
 9   married              10000 non-null  float64
 10  children             10000 non-null  float64
 11  postal_code          10000 non-null  int64  
 12  annual_mileage       9043 non-null   float64
 13  vehicle_type         10000 non-null  object 
 14  speeding_violations  10000 non-null  int64  
 15  duis                 10000 non-null  

Filling Missing Values

In [33]:
# Fill missing values with the median for numerical columns and mode for categorical columns
for column in car_insurance.columns:
    if car_insurance[column].dtype == "object":
        car_insurance[column].fillna(car_insurance[column].mode()[0], inplace=True)
    else:
        car_insurance[column].fillna(car_insurance[column].median(), inplace=True)

# Verify that missing values are handled
print(car_insurance.isnull().sum())

# Display summary statistics
print(car_insurance.describe())

# Display the first few rows to get an overview
print(car_insurance.head())

id                     0
age                    0
gender                 0
driving_experience     0
education              0
income                 0
credit_score           0
vehicle_ownership      0
vehicle_year           0
married                0
children               0
postal_code            0
annual_mileage         0
vehicle_type           0
speeding_violations    0
duis                   0
past_accidents         0
outcome                0
dtype: int64
                  id           age  ...  past_accidents       outcome
count   10000.000000  10000.000000  ...    10000.000000  10000.000000
mean   500521.906800      1.489500  ...        1.056300      0.313300
std    290030.768758      1.025278  ...        1.652454      0.463858
min       101.000000      0.000000  ...        0.000000      0.000000
25%    249638.500000      1.000000  ...        0.000000      0.000000
50%    501777.000000      1.000000  ...        0.000000      0.000000
75%    753974.500000      2.000000  ...        

Preparing for Modeling

In [34]:
# Define the target variable
y = car_insurance["outcome"]

# Prepare a dictionary to store feature accuracies
feature_accuracies = {}

# Define features excluding 'id' and 'outcome'
features = [col for col in car_insurance.columns if col not in ["id", "outcome"]]

Building and Storing the models

In [35]:
# Initialize the list to store trained models
trained_models = []

# Train a logistic regression model for each feature
for feature in features:
    X = car_insurance[[feature]]
    
    # Convert non-numeric features to numeric using one-hot encoding
    if X[feature].dtype == 'object':
        X = pd.get_dummies(X, columns=[feature], drop_first=True)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    trained_models.append((feature, model, X_test, y_test))

Measuring performance by calculating the accuracy of each model

In [36]:
# Prepare a list to store model accuracies and metrics
model_results = []

# Measure performance of each trained model
for feature, model, X_test, y_test in trained_models:
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Create confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    
    # Store results
    model_results.append({
        "Feature": feature,
        "Accuracy": accuracy,
        "True Negatives": tn,
        "False Positives": fp,
        "False Negatives": fn,
        "True Positives": tp
    })

# Convert results to DataFrame
model_results_df = pd.DataFrame(model_results)

# Display sorted results
model_results_df = model_results_df.sort_values(by="Accuracy", ascending=False)
print(model_results_df)

                Feature  Accuracy  ...  False Negatives  True Positives
0                   age    0.7830  ...              334             299
2    driving_experience    0.7780  ...              188             445
4                income    0.7440  ...              388             245
6     vehicle_ownership    0.7280  ...              291             342
5          credit_score    0.6985  ...              501             132
1                gender    0.6835  ...              633               0
3             education    0.6835  ...              633               0
7          vehicle_year    0.6835  ...              633               0
8               married    0.6835  ...              633               0
9              children    0.6835  ...              633               0
10          postal_code    0.6835  ...              633               0
11       annual_mileage    0.6835  ...              633               0
12         vehicle_type    0.6835  ...              633         

Finding the Best Performing Model

In [37]:
# Finding the best performing model
best_model = model_results_df.iloc[0]  # Select the row with the highest accuracy

# Create a DataFrame with the best feature and accuracy
best_feature_df = pd.DataFrame({
    "best_feature": [str(best_model["Feature"])],
    "best_accuracy": [best_model["Accuracy"]]
})

# Display the best feature
print(best_feature_df)

  best_feature  best_accuracy
0          age          0.783
